In [1]:
import pandas as pd

pd.options.display.max_columns = 999

In [2]:
df = pd.read_parquet('/home/franciscofoz/Documents/GitHub/recomendador-artigos-OpenAlex-GPT/df_concatenado.parquet')
df.head()

,doi,title,publication_date,open_access,concepts,Environmental science,Psychology,Computer science,Art,Business,Mathematics,Geography,Biology,Political science,Philosophy,History,Medicine,Physics,Materials science,Economics,Engineering,Sociology,Chemistry,Geology
0,https://doi.org/10.5281/zenodo.8403288,Performance Evaluation of DSTATCOM for 180 km ...,2023-10-03,True,Environmental science,0.3234,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://doi.org/10.5281/zenodo.8403717,MMA: 10 Facebook Pages to Follow About Cage Fi...,2023-10-03,True,Psychology;Computer science;Art;Business;Mathe...,0.0000,0.3502,0.3296,0.3167,0.1581,0.0887,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://doi.org/10.5281/zenodo.8403719,KICHIK BIZNES MUHITI VA UNING TUZILISHI,2023-10-03,True,Computer science,0.0000,0.0000,0.2757,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://doi.org/10.5281/zenodo.8400069,Neoextractivismo en Bolivia. Sostenibilidad e ...,2023-10-02,True,Geography,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://doi.org/10.5281/zenodo.8403684,TUTORIA HUMANIZADA: A UTILIZAÇÃO DE METODOLOG...,2023-10-03,True,Art,0.0000,0.0000,0.0000,0.3220,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
def filtrar_dataframe_por_termos(df, termos):
    
    termos_formatados = [f"`{termo}`" if ' ' in termo else termo for termo in termos]
    
    filtro = " or ".join(f"{termo} > 0" for termo in termos_formatados)
    
    df_filtrado = df.query(filtro).loc[:, ['doi', 'title', 'publication_date', 'open_access'] + termos]
    
    return df_filtrado

def filtrar_dataframe_por_acesso_aberto(df,resposta):
    
    if resposta == 'Sim':
        df = df.query('open_access == True')
    
    return df

def criar_coluna_score(df):
    
    df['score'] = df.iloc[:,4:].sum(axis=1)
    
    return df

def atribuir_fator_termo_score(df, termos):
    df_copy = df.copy()
    
    df_copy['title'] = df_copy['title'].fillna('')
    
    for term in termos:
        mask = df_copy['title'].str.upper().str.contains(term.upper())
        df_copy.loc[mask, 'score'] *= 2
    
    df_copy = df_copy.sort_values(by='score', ascending=False)
    
    return df_copy

def atribuir_fator_termo_similar_score(df, termos_similares):
    df_copy = df.copy()
    
    df_copy['title'] = df_copy['title'].fillna('')
    
    for term in termos_similares:
        mask = df_copy['title'].str.upper().str.contains(term.upper())
        df_copy.loc[mask, 'score'] *= 1.5
    
    df_copy = df_copy.sort_values(by='score', ascending=False)
    
    return df_copy

def normalizar_score(df):
    min_score = df['score'].min()
    max_score = df['score'].max()


    df['score_normalizado'] = ((df['score'] - min_score) / (max_score - min_score))

    return df

In [4]:
df

,doi,title,publication_date,open_access,concepts,Environmental science,Psychology,Computer science,Art,Business,Mathematics,Geography,Biology,Political science,Philosophy,History,Medicine,Physics,Materials science,Economics,Engineering,Sociology,Chemistry,Geology
0,https://doi.org/10.5281/zenodo.8403288,Performance Evaluation of DSTATCOM for 180 km ...,2023-10-03,True,Environmental science,0.3234,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
1,https://doi.org/10.5281/zenodo.8403717,MMA: 10 Facebook Pages to Follow About Cage Fi...,2023-10-03,True,Psychology;Computer science;Art;Business;Mathe...,0.0000,0.3502,0.3296,0.3167,0.1581,0.0887,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
2,https://doi.org/10.5281/zenodo.8403719,KICHIK BIZNES MUHITI VA UNING TUZILISHI,2023-10-03,True,Computer science,0.0000,0.0000,0.2757,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
3,https://doi.org/10.5281/zenodo.8400069,Neoextractivismo en Bolivia. Sostenibilidad e ...,2023-10-02,True,Geography,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3049,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
4,https://doi.org/10.5281/zenodo.8403684,TUTORIA HUMANIZADA: A UTILIZAÇÃO DE METODOLOG...,2023-10-03,True,Art,0.0000,0.0000,0.0000,0.3220,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35322,https://doi.org/10.1117/12.2679007,3D imaging of pulsed laser ablation at 1 milli...,2023-10-04,False,Materials science;Engineering;Physics;Computer...,0.0000,0.0000,0.3352,0.1411,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.2392,0.4558,0.0,0.1686,0.0000,0.0000,0.0
35323,https://doi.org/10.47134/researchjet.v3i1.12,Dilema Perizinan Alih Fungsi Bangunan Rumah Di...,2023-10-02,True,Computer science;Philosophy;Sociology;Business...,0.0000,0.0000,0.0749,0.0000,0.3310,0.0000,0.0000,0.0000,0.5423,0.0984,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.2483,0.0000,0.0
35324,https://doi.org/10.1101/2023.10.01.560349,SaProt: Protein Language Modeling with Structu...,2023-10-02,True,Engineering;Biology;Computer science,0.0000,0.0000,0.7159,0.0000,0.0000,0.0000,0.0000,0.1143,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0726,0.0000,0.0000,0.0
35325,https://doi.org/10.3389/fmicb.2023.1275849,Mechanism of action of sprG1-encoded type I to...,2023-10-03,True,Biology;Chemistry,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5394,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.3713,0.0


In [5]:
df.shape[0]

35327

In [33]:
df_filtrado = filtrar_dataframe_por_termos(df,['Psychology'])

df_filtrado = filtrar_dataframe_por_acesso_aberto(df_filtrado,'Não')

df_filtrado = criar_coluna_score(df_filtrado)

df_filtrado = atribuir_fator_termo_score(df_filtrado,['Alzheimer'])

df_filtrado = atribuir_fator_termo_similar_score(df_filtrado,
                                                 ['Alzheimer', 'Dementia', 'Cognitive Decline', 'Memory Loss', 'Neurodegenerative Disease', 'Alzheimer', 'Demência', 'Declínio Cognitivo', 'Perda de Memória', 'Doença Neurodegenerativa'])

df_filtrado = normalizar_score(df_filtrado)

print(df_filtrado.shape[0],'\t',df_filtrado.shape[0]/df.shape[0])
df_filtrado.head(10)

4490 	 0.12709825346052595


,doi,title,publication_date,open_access,Psychology,score,score_normalizado
29405,https://doi.org/10.1017/s1478951523001360,Burden and anticipatory grief in caregivers of...,2023-10-05,True,0.6845,4.620375,1.000000
29950,https://doi.org/10.5281/zenodo.8398219,DEPRESSION OR DEMENTIA IS RELATED MORE TO ALZH...,2023-10-02,True,0.4174,2.817450,0.606389
12638,https://doi.org/10.36227/techrxiv.24217041.v1,Analyzing and Comparing the Brain Activity of ...,2023-10-02,True,0.5922,2.664900,0.573085
12692,https://doi.org/10.36227/techrxiv.24217041,Analyzing and Comparing the Brain Activity of ...,2023-10-02,True,0.5652,2.543400,0.546559
24642,https://doi.org/10.1101/2023.10.02.23296420,The Subcortical Default Mode Network and Alzhe...,2023-10-02,True,0.5382,2.421900,0.520033
10255,https://doi.org/10.1212/wnl.0000000000207840,"Editors' Note: Estrogen Receptor Genes, Cognit...",2023-10-02,False,0.3437,2.319975,0.497781
10256,https://doi.org/10.1212/wnl.0000000000207842,"Author Response: Estrogen Receptor Genes, Cogn...",2023-10-02,False,0.3251,2.194425,0.470372
3630,https://doi.org/10.3233/jad-230732,Informing Disclosure: Efficacy of a Brief Educ...,2023-10-03,False,0.4766,2.144700,0.459516
29410,https://doi.org/10.1002/brb3.3271,Voice analysis results in individuals with Alz...,2023-10-04,True,0.4670,2.101500,0.450084
19935,https://doi.org/10.3389/fnhum.2023.1234168,Identifying biomarkers for tDCS treatment resp...,2023-10-04,True,0.4327,1.947150,0.416387


In [34]:
df_filtrado

,doi,title,publication_date,open_access,Psychology,score,score_normalizado
29405,https://doi.org/10.1017/s1478951523001360,Burden and anticipatory grief in caregivers of...,2023-10-05,True,0.6845,4.620375,1.000000
29950,https://doi.org/10.5281/zenodo.8398219,DEPRESSION OR DEMENTIA IS RELATED MORE TO ALZH...,2023-10-02,True,0.4174,2.817450,0.606389
12638,https://doi.org/10.36227/techrxiv.24217041.v1,Analyzing and Comparing the Brain Activity of ...,2023-10-02,True,0.5922,2.664900,0.573085
12692,https://doi.org/10.36227/techrxiv.24217041,Analyzing and Comparing the Brain Activity of ...,2023-10-02,True,0.5652,2.543400,0.546559
24642,https://doi.org/10.1101/2023.10.02.23296420,The Subcortical Default Mode Network and Alzhe...,2023-10-02,True,0.5382,2.421900,0.520033
...,...,...,...,...,...,...,...
17817,https://doi.org/10.25681/iaras.0130-2620.271.4...,THE MOSCOW UNIT OF THE RAIMK AND THE PROJECT O...,2023-10-03,True,0.0568,0.056800,0.003690
9885,https://doi.org/10.1158/1055-9965.24229768,Supplementary Table 3 from The Association of ...,2023-10-02,True,0.0531,0.053100,0.002882
8615,https://doi.org/10.1155/2023/9863498,Retracted: Blended Teaching Strategies of Coll...,2023-10-04,True,0.0512,0.051200,0.002467
7690,https://doi.org/10.47361/2542-0259-2023-3-35-6...,Interstate standardization of antimonopoly com...,2023-10-02,False,0.0486,0.048600,0.001899


In [17]:
def normalizar_score(df):
    min_score = df['score'].min()
    max_score = df['score'].max()


    df['score_normalized'] = ((df['score'] - min_score) / (max_score - min_score)) * (0.99 - 0.1) + 0.1

    return df

In [18]:
df_filtrado

,doi,title,publication_date,open_access,Psychology,score,score_normalizado
29405,https://doi.org/10.1017/s1478951523001360,Burden and anticipatory grief in caregivers of...,2023-10-05,True,0.6845,4.620375,0.990000
29950,https://doi.org/10.5281/zenodo.8398219,DEPRESSION OR DEMENTIA IS RELATED MORE TO ALZH...,2023-10-02,True,0.4174,2.817450,0.639686
12638,https://doi.org/10.36227/techrxiv.24217041.v1,Analyzing and Comparing the Brain Activity of ...,2023-10-02,True,0.5922,2.664900,0.610045
12692,https://doi.org/10.36227/techrxiv.24217041,Analyzing and Comparing the Brain Activity of ...,2023-10-02,True,0.5652,2.543400,0.586438
24642,https://doi.org/10.1101/2023.10.02.23296420,The Subcortical Default Mode Network and Alzhe...,2023-10-02,True,0.5382,2.421900,0.562830
...,...,...,...,...,...,...,...
17817,https://doi.org/10.25681/iaras.0130-2620.271.4...,THE MOSCOW UNIT OF THE RAIMK AND THE PROJECT O...,2023-10-03,True,0.0568,0.056800,0.103284
9885,https://doi.org/10.1158/1055-9965.24229768,Supplementary Table 3 from The Association of ...,2023-10-02,True,0.0531,0.053100,0.102565
8615,https://doi.org/10.1155/2023/9863498,Retracted: Blended Teaching Strategies of Coll...,2023-10-04,True,0.0512,0.051200,0.102196
7690,https://doi.org/10.47361/2542-0259-2023-3-35-6...,Interstate standardization of antimonopoly com...,2023-10-02,False,0.0486,0.048600,0.101690


In [16]:
max_score

4.620375000000001

In [170]:
for i in df_filtrado.title:
    print(i)


Burden and anticipatory grief in caregivers of family members with Alzheimer’s disease and other dementias
DEPRESSION OR DEMENTIA IS RELATED MORE TO ALZHEIMER'S DISEASE (AD)
Analyzing and Comparing the Brain Activity of Alzheimer's Patients with Healthy Subjects
Analyzing and Comparing the Brain Activity of Alzheimer's Patients with Healthy Subjects
The Subcortical Default Mode Network and Alzheimer Disease: A systematic review and Activation Likelihood Estimation Meta-Analysis
Editors' Note: Estrogen Receptor Genes, Cognitive Decline, and Alzheimer Disease
Author Response: Estrogen Receptor Genes, Cognitive Decline, and Alzheimer Disease
Informing Disclosure: Efficacy of a Brief Educational Intervention on Research Participants’ Knowledge about Alzheimer’s Disease Biomarkers
Voice analysis results in individuals with Alzheimer's disease: How do age and cognitive status affect voice parameters?
Identifying biomarkers for tDCS treatment response in Alzheimer’s disease patients: a machin

In [198]:
#PROMPT:

'''

A partir desse termo ['Alzheimer']. 

Gere 5 termos semelhantes em inglês e 5 em português. 

Responda com uma única lista Python.

Como nesse exemplo: 
['"Artificial Intelligence","Inteligência Artificial"]

Não responda mais nada além da lista.

'''

'\n\nA partir desse termo [\'Alzheimer\']. \n\nGere 5 termos semelhantes em inglês e 5 em português. \n\nResponda com uma única lista Python.\n\nComo nesse exemplo: \n[\'"Artificial Intelligence","Inteligência Artificial"]\n\nNão responda mais nada além da lista.\n\n'